In [27]:
# open each file in "results"

path = "results"
import os
files = os.listdir(path)
files = [os.path.join(path, f) for f in files]

rows = []

for file in files:
    handle = open(file, "rt")
    id = file.split("/")[-1].split(".")[0]
    for line in handle:
        columns = line.strip().split("\t")
        pos = int(columns[0])
        counts = [int(c) for c in columns[1:]]
        id = file.split("/")[-1].split(".")[0]
        row = {"id": id, "pos": pos, "A": counts[0], "C": counts[1], "G": counts[2], "T": counts[3]}
        rows.append(row)

import pandas as pd
df = pd.DataFrame(rows)




In [28]:
file = "./sra_metadata.csv"
cols = ["id","biosample","sample_name","drug"]
# no header, usecols to select columns
meta = pd.read_csv(file, header=None, usecols=[0,1,2,3], names=cols)
# sample name is study_name/detailed_sample_name/country, split these up
meta["study_name"] = meta["sample_name"].str.split("/").str[0]
meta["detailed_sample_name"] = meta["sample_name"].str.split("/").str[1]
meta["country"] = meta["sample_name"].str.split("/").str[2]

# detailed sample name is individualid_day, split these up
meta["individualid"] = meta["detailed_sample_name"].str.split("_").str[0]
meta["day"] = meta["detailed_sample_name"].str.split("_").str[1]

# drop detailed sample name
meta = meta.drop(columns=["detailed_sample_name"])

In [29]:

# find positions that occur more than once and remove them

df.groupby("pos").filter(lambda x: len(x) == 1)

def create_string_from_row(row):
    total = sum(row[["A", "C", "G", "T"]])
    ratioed = row[["A", "C", "G", "T"]]/total
    # print any bases that are > 0.1
    bases = ["A", "C", "G", "T"]
    bases = [b for b in bases if ratioed[b] > 0.1]
    return "".join(bases)

df["bases"] = df.apply(create_string_from_row, axis=1)

df = df.merge(meta, on="id")

        

In [30]:
d5 = df[df['day']=='d5']
# aggregate "bases" by "drug"
d5.groupby("drug")["bases"].value_counts()

drug          bases
molnupiravir  CT       293
              AG       217
              GT       141
              AC        28
              CG        17
              AT        16
              ACT        1
placebo       GT       121
              CT       104
              AG        67
              AC        21
              AT        18
              CG        14
              AGT        1
Name: bases, dtype: int64

In [ ]:
from Bio import SeqIO

# read in the reference sequence
ref = SeqIO.read("data/NC_045512.2.fasta", "fasta")